In [1]:
import os
import numpy as np
import random as rn
import tensorflow as tf

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(7)
rn.seed(7)

session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1,
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)

from keras import backend as K

tf.set_random_seed(7)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

# import some library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import itertools
import time
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam, Adamax, Adagrad
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.read_csv('../data/feature_selection_negative.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [3]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
def create_model(optimizer='rmsprop'):
    tf.set_random_seed(42)
    c=0.4
    # make keras model
    start = time.time()
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(1024, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    
    return model

In [5]:
model = KerasClassifier(
    build_fn=create_model, 
    verbose=1, 
    epochs=100, 
    batch_size=1000
)

In [6]:
p = [
        'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 
        'Adam', 'Adamax', 'Nadam'
]  # optimizers

params = dict(
    optimizer=p
             )

In [7]:
grid = GridSearchCV(
    estimator=model, 
    param_grid=params
)

In [8]:
grid_result = grid.fit(X_train, y_train_for_keras)

Epoch 1/100
248/248 [==============================] - 1s 4ms/step - loss: 2.0250 - acc: 0.2460
Epoch 2/100
248/248 [==============================] - 0s 45us/step - loss: 2.0321 - acc: 0.2419
Epoch 3/100
248/248 [==============================] - 0s 42us/step - loss: 1.8597 - acc: 0.2823
Epoch 4/100
248/248 [==============================] - 0s 44us/step - loss: 1.8119 - acc: 0.2823
Epoch 5/100
248/248 [==============================] - 0s 43us/step - loss: 1.9876 - acc: 0.3306
Epoch 6/100
248/248 [==============================] - 0s 43us/step - loss: 1.6780 - acc: 0.3669
Epoch 7/100
248/248 [==============================] - 0s 44us/step - loss: 1.7007 - acc: 0.3347
Epoch 8/100
248/248 [==============================] - 0s 49us/step - loss: 1.6399 - acc: 0.3306
Epoch 9/100
248/248 [==============================] - 0s 45us/step - loss: 1.5600 - acc: 0.3468
Epoch 10/100
248/248 [==============================] - 0s 46us/step - loss: 1.6894 - acc: 0.3952
Epoch 11/100
248/248 [========

Epoch 85/100
248/248 [==============================] - 0s 49us/step - loss: 0.9521 - acc: 0.5806
Epoch 86/100
248/248 [==============================] - 0s 52us/step - loss: 0.9474 - acc: 0.6008
Epoch 87/100
248/248 [==============================] - 0s 48us/step - loss: 0.9564 - acc: 0.5806
Epoch 88/100
248/248 [==============================] - 0s 45us/step - loss: 1.0311 - acc: 0.5726
Epoch 89/100
248/248 [==============================] - 0s 48us/step - loss: 0.8885 - acc: 0.6169
Epoch 90/100
248/248 [==============================] - 0s 44us/step - loss: 0.9808 - acc: 0.6048
Epoch 91/100
248/248 [==============================] - 0s 48us/step - loss: 1.0126 - acc: 0.5726
Epoch 92/100
248/248 [==============================] - 0s 46us/step - loss: 0.9849 - acc: 0.6129
Epoch 93/100
248/248 [==============================] - 0s 47us/step - loss: 1.0156 - acc: 0.5484
Epoch 94/100
248/248 [==============================] - 0s 50us/step - loss: 0.9901 - acc: 0.5847
Epoch 95/100
248/248

Epoch 68/100
249/249 [==============================] - 0s 49us/step - loss: 1.0759 - acc: 0.5462
Epoch 69/100
249/249 [==============================] - 0s 48us/step - loss: 1.1808 - acc: 0.5462
Epoch 70/100
249/249 [==============================] - 0s 47us/step - loss: 1.0793 - acc: 0.5422
Epoch 71/100
249/249 [==============================] - 0s 43us/step - loss: 1.0578 - acc: 0.5422
Epoch 72/100
249/249 [==============================] - 0s 43us/step - loss: 1.0442 - acc: 0.5622
Epoch 73/100
249/249 [==============================] - 0s 47us/step - loss: 1.0684 - acc: 0.5542
Epoch 74/100
249/249 [==============================] - 0s 49us/step - loss: 1.0668 - acc: 0.5542
Epoch 75/100
249/249 [==============================] - 0s 49us/step - loss: 1.0047 - acc: 0.5783
Epoch 76/100
249/249 [==============================] - 0s 46us/step - loss: 1.0483 - acc: 0.5382
Epoch 77/100
249/249 [==============================] - 0s 45us/step - loss: 0.9635 - acc: 0.6064
Epoch 78/100
249/249

Epoch 51/100
249/249 [==============================] - 0s 52us/step - loss: 1.1078 - acc: 0.5261
Epoch 52/100
249/249 [==============================] - 0s 50us/step - loss: 1.0215 - acc: 0.5663
Epoch 53/100
249/249 [==============================] - 0s 52us/step - loss: 1.1386 - acc: 0.5181
Epoch 54/100
249/249 [==============================] - 0s 49us/step - loss: 1.0624 - acc: 0.5141
Epoch 55/100
249/249 [==============================] - 0s 49us/step - loss: 1.1543 - acc: 0.5502
Epoch 56/100
249/249 [==============================] - 0s 49us/step - loss: 1.0410 - acc: 0.5382
Epoch 57/100
249/249 [==============================] - 0s 53us/step - loss: 1.0265 - acc: 0.5502
Epoch 58/100
249/249 [==============================] - 0s 47us/step - loss: 1.2061 - acc: 0.4819
Epoch 59/100
249/249 [==============================] - 0s 48us/step - loss: 1.0717 - acc: 0.5301
Epoch 60/100
249/249 [==============================] - 0s 43us/step - loss: 1.0211 - acc: 0.5663
Epoch 61/100
249/249

Epoch 34/100
248/248 [==============================] - 0s 70us/step - loss: 0.5119 - acc: 0.7984
Epoch 35/100
248/248 [==============================] - 0s 62us/step - loss: 0.5622 - acc: 0.7702
Epoch 36/100
248/248 [==============================] - 0s 54us/step - loss: 0.5386 - acc: 0.7702
Epoch 37/100
248/248 [==============================] - 0s 50us/step - loss: 0.5546 - acc: 0.7863
Epoch 38/100
248/248 [==============================] - 0s 59us/step - loss: 0.4663 - acc: 0.7984
Epoch 39/100
248/248 [==============================] - 0s 62us/step - loss: 0.5609 - acc: 0.8024
Epoch 40/100
248/248 [==============================] - 0s 63us/step - loss: 0.6107 - acc: 0.7500
Epoch 41/100
248/248 [==============================] - 0s 56us/step - loss: 0.6202 - acc: 0.7944
Epoch 42/100
248/248 [==============================] - 0s 54us/step - loss: 0.5239 - acc: 0.7702
Epoch 43/100
248/248 [==============================] - 0s 65us/step - loss: 0.4770 - acc: 0.8024
Epoch 44/100
248/248

...

Epoch 31/100
248/248 [==============================] - 0s 67us/step - loss: 0.4788 - acc: 0.8266
Epoch 32/100
248/248 [==============================] - 0s 70us/step - loss: 0.4503 - acc: 0.8105
Epoch 33/100
248/248 [==============================] - 0s 67us/step - loss: 0.4347 - acc: 0.8347
Epoch 34/100
248/248 [==============================] - 0s 78us/step - loss: 0.4334 - acc: 0.8185
Epoch 35/100
248/248 [==============================] - 0s 69us/step - loss: 0.4661 - acc: 0.8024
Epoch 36/100
248/248 [==============================] - 0s 66us/step - loss: 0.4369 - acc: 0.8105
Epoch 37/100
248/248 [==============================] - 0s 68us/step - loss: 0.4036 - acc: 0.8508
Epoch 38/100
248/248 [==============================] - 0s 69us/step - loss: 0.3701 - acc: 0.8468
Epoch 39/100
248/248 [==============================] - 0s 65us/step - loss: 0.3759 - acc: 0.8427
Epoch 40/100
248/248 [==============================] - 0s 65us/step - loss: 0.3934 - acc: 0.8589
Epoch 41/100
248/248

Epoch 14/100
249/249 [==============================] - 0s 69us/step - loss: 0.7476 - acc: 0.6867
Epoch 15/100
249/249 [==============================] - 0s 63us/step - loss: 0.7304 - acc: 0.7028
Epoch 16/100
249/249 [==============================] - 0s 64us/step - loss: 0.6845 - acc: 0.7349
Epoch 17/100
249/249 [==============================] - 0s 73us/step - loss: 0.6083 - acc: 0.7390
Epoch 18/100
249/249 [==============================] - 0s 74us/step - loss: 0.6690 - acc: 0.7390
Epoch 19/100
249/249 [==============================] - 0s 64us/step - loss: 0.7062 - acc: 0.7269
Epoch 20/100
249/249 [==============================] - 0s 65us/step - loss: 0.6383 - acc: 0.7550
Epoch 21/100
249/249 [==============================] - 0s 65us/step - loss: 0.5502 - acc: 0.7590
Epoch 22/100
249/249 [==============================] - 0s 64us/step - loss: 0.5748 - acc: 0.7791
Epoch 23/100
249/249 [==============================] - 0s 64us/step - loss: 0.5680 - acc: 0.7711
Epoch 24/100
249/249

Epoch 98/100
249/249 [==============================] - 0s 77us/step - loss: 0.1669 - acc: 0.9438
Epoch 99/100
249/249 [==============================] - 0s 71us/step - loss: 0.1505 - acc: 0.9438
Epoch 100/100
249/249 [==============================] - 0s 24us/step
Epoch 1/100
249/249 [==============================] - 4s 16ms/step - loss: 2.0139 - acc: 0.2329
Epoch 2/100
249/249 [==============================] - 0s 63us/step - loss: 2.4839 - acc: 0.4137
Epoch 3/100
249/249 [==============================] - 0s 75us/step - loss: 1.7230 - acc: 0.4257
Epoch 4/100
249/249 [==============================] - 0s 72us/step - loss: 1.7210 - acc: 0.4940
Epoch 5/100
249/249 [==============================] - 0s 74us/step - loss: 1.7060 - acc: 0.4980
Epoch 6/100
249/249 [==============================] - 0s 64us/step - loss: 1.7842 - acc: 0.5141
Epoch 7/100
249/249 [==============================] - 0s 64us/step - loss: 1.0669 - acc: 0.5422
Epoch 8/100
249/249 [==============================] - 

Epoch 81/100
249/249 [==============================] - 0s 76us/step - loss: 0.1801 - acc: 0.9317
Epoch 82/100
249/249 [==============================] - 0s 99us/step - loss: 0.1840 - acc: 0.9277
Epoch 83/100
249/249 [==============================] - 0s 71us/step - loss: 0.1780 - acc: 0.9197
Epoch 84/100
249/249 [==============================] - 0s 72us/step - loss: 0.1743 - acc: 0.9518
Epoch 85/100
249/249 [==============================] - 0s 67us/step - loss: 0.1512 - acc: 0.9438
Epoch 86/100
249/249 [==============================] - 0s 70us/step - loss: 0.1524 - acc: 0.9398
Epoch 87/100
249/249 [==============================] - 0s 68us/step - loss: 0.1601 - acc: 0.9277
Epoch 88/100
249/249 [==============================] - 0s 67us/step - loss: 0.1504 - acc: 0.9398
Epoch 89/100
249/249 [==============================] - 0s 65us/step - loss: 0.1436 - acc: 0.9598
Epoch 90/100
249/249 [==============================] - 0s 75us/step - loss: 0.1405 - acc: 0.9558
Epoch 91/100
249/249

Epoch 64/100
373/373 [==============================] - 0s 51us/step - loss: 0.4340 - acc: 0.8257
Epoch 65/100
373/373 [==============================] - 0s 50us/step - loss: 0.5047 - acc: 0.7936
Epoch 66/100
373/373 [==============================] - 0s 51us/step - loss: 0.4660 - acc: 0.7962
Epoch 67/100
373/373 [==============================] - 0s 59us/step - loss: 0.5156 - acc: 0.7614
Epoch 68/100
373/373 [==============================] - 0s 55us/step - loss: 0.4680 - acc: 0.7882
Epoch 69/100
373/373 [==============================] - 0s 54us/step - loss: 0.4315 - acc: 0.8150
Epoch 70/100
373/373 [==============================] - 0s 52us/step - loss: 0.4761 - acc: 0.8016
Epoch 71/100
373/373 [==============================] - 0s 51us/step - loss: 0.4309 - acc: 0.8177
Epoch 72/100
373/373 [==============================] - 0s 55us/step - loss: 0.4440 - acc: 0.8177
Epoch 73/100
373/373 [==============================] - 0s 62us/step - loss: 0.4251 - acc: 0.8257
Epoch 74/100
373/373

In [9]:
t = grid_result.best_estimator_

In [10]:
t.score(X_test, y_test)

94/94 [==============================] - 1s 13ms/step


0.7127659320831299

In [11]:
t.model.save('../model/Keras_best_params_neg_fs.h5')